In [2]:
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from scipy import io as sio
from tensorflow.python.framework import ops
from dfs2 import DeepFeatureSelectionNew
import numpy as np
from sklearn.datasets import make_classification
from sklearn.preprocessing import normalize
from sklearn.preprocessing import Imputer

data_All = sio.loadmat("/Volumes/TONY/LabWW/Alzheimer/GSE44772/GSE44772_data.mat")

data_PFC = sio.loadmat("/Volumes/TONY/LabWW/Alzheimer/GSE44772/GSE44772_PFC.mat")
data_VC = sio.loadmat("/Volumes/TONY/LabWW/Alzheimer/GSE44772/GSE44772_VC.mat")
data_CR = sio.loadmat("/Volumes/TONY/LabWW/Alzheimer/GSE44772/GSE44772_CR.mat")

In [6]:
data_All['ID']

array([[          1, 10019475365],
       [          2, 10019481149],
       [          3, 10019495284],
       ..., 
       [      39278, 10033669047],
       [      39279, 10033669048],
       [      39280, 10033669049]])

In [7]:
data_VC['X'].shape

(230, 39280)

In [7]:
def setRegion(region):
    if region == 'PFC':
        ourdata = data_PFC
    elif region == 'VC':
        ourdata = data_VC
    elif region == 'CR':
        ourdata = data_CR
    elif region == 'All':
        ourdata = data_All
        
    inputX = ourdata['X']
    # Inpute using median
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    imp.fit(inputX)
    inputX = imp.transform(inputX)
    
    inputY = ourdata['y'][0,:]
    
    return inputX, inputY

In [8]:
inputX, inputY = setRegion('CR')

In [9]:
ig(inputX, inputY)

ValueError: Input X must be non-negative.

In [22]:
# X_train, X_test, y_train, y_test = train_test_split(inputX, inputY, test_size=0.2, random_state=1)

dfsMLP = DeepFeatureSelectionNew(X_train, X_test, y_train, y_test, n_input=1, hidden_dims=[100], dropout=[False], \
                                 learning_rate=0.01, lambda1=0.001, lambda2=1, alpha1=0.1, alpha2=0, activation='tanh', weight_init='uniform', \
                                 epochs=100, optimizer='Adam', print_step=10)

dfsMLP.train(batch_size=200)

epoch 0: global loss = 44.2518844604
('Train accuracy:', 0.82608694)
('Test accuracy:', 0.76086956)
epoch 10: global loss = 10.9437866211
('Train accuracy:', 0.93478262)
('Test accuracy:', 0.84782606)
epoch 20: global loss = 4.2828540802
('Train accuracy:', 0.95108694)
('Test accuracy:', 0.84782606)
epoch 30: global loss = 1.71155512333
('Train accuracy:', 0.97826087)
('Test accuracy:', 0.84782606)
epoch 40: global loss = 0.955415606499
('Train accuracy:', 0.98369563)
('Test accuracy:', 0.84782606)
epoch 50: global loss = 0.666642308235
('Train accuracy:', 0.98913044)
('Test accuracy:', 0.84782606)
epoch 60: global loss = 0.524983823299
('Train accuracy:', 0.98913044)
('Test accuracy:', 0.84782606)
epoch 70: global loss = 0.449758648872
('Train accuracy:', 0.98913044)
('Test accuracy:', 0.84782606)
epoch 80: global loss = 0.407711446285
('Train accuracy:', 0.98913044)
('Test accuracy:', 0.84782606)
epoch 90: global loss = 0.38348698616
('Train accuracy:', 0.98913044)
('Test accuracy:',

In [42]:
from sklearn.cross_validation import cross_val_score
from sklearn.svm import LinearSVC

svm = LinearSVC()

# All cross validation 0.900105526433
# PFC cross validation 0.90895672731
# VC cross validation 0.913401171755
# CR cross validation 0.904520505705

# keke, keke_test = topDFSfeatures(0,150)
scores = cross_val_score(svm, inputX, inputY, cv=5)
print(np.mean(scores))

0.904520505705


In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb

rf = RandomForestClassifier(criterion="entropy", n_estimators = 300, max_depth = 5)
for region in ['All','CR','PFC','VC']:
    inputX, inputY = setRegion(region)
    scores = cross_val_score(rf, inputX, inputY, cv=5)
    print(region + "cross validation result:" + str(np.mean(scores)))

# gbm = xgb.XGBClassifier(max_depth=5, n_estimators=400, learning_rate=0.05).fit(top_train, y_train)
# y_pred = gbm.predict(top_test)

# featurescores = gbm.feature_importances_

# print(accuracy_score(y_test, y_pred))

Allcross validation result:0.881546578956
CRcross validation result:0.891380409086
PFCcross validation result:0.900265186556
VCcross validation result:0.878517833282


In [66]:
from sklearn.cross_validation import KFold

# All XGBoost cross validation result:0.882608695652

for region in ['CR','PFC','VC']:
    inputX, inputY = setRegion(region)
    scores = []
    for train_index, test_index in kf:
        kf = KFold(n=inputX.shape[0], n_folds=5, shuffle=True, random_state=44)
        X_train, X_test, y_train, y_test = inputX[train_index], inputX[test_index], inputY[train_index], inputY[test_index]
        gbm = xgb.XGBClassifier(max_depth=5, n_estimators=300, learning_rate=0.05).fit(X_train, y_train)
        y_pred = gbm.predict(X_test)
    #     featurescores = gbm.feature_importances_
        scores.append(accuracy_score(y_test, y_pred))
    print(region + " XGBoost cross validation result:" + str(np.mean(scores)))

CR XGBoost cross validation result:0.882608695652
PFC XGBoost cross validation result:0.882608695652
VC XGBoost cross validation result:0.886956521739


In [63]:
kf = KFold(n=inputX.shape[0], n_folds=5, shuffle=True, random_state=44)

kf

1. MI
2. Top 100
3. GO analysis, pathway analysis

sklearn.cross_validation.KFold(n=230, n_folds=5, shuffle=True, random_state=44)

In [65]:
for train_index, test_index in kf:
    print test_index

[  3   5   7   9  23  30  35  43  44  46  51  54  58  61  65  69  73  79
  82  85  88 102 105 114 115 133 135 137 142 160 162 174 176 181 183 186
 187 192 193 204 207 209 211 213 220 223]
[  2   6  12  13  17  19  24  28  32  37  38  39  45  47  48  60  64  68
  77  81  94  95 108 111 125 129 134 136 138 139 147 152 153 154 156 158
 159 167 171 175 197 205 208 214 222 229]
[  0  15  26  27  33  36  41  52  70  71  75  76  80  87  89  90  99 106
 107 110 112 113 118 124 126 127 148 149 155 157 165 166 169 170 177 178
 180 182 184 185 194 212 215 218 219 228]
[  4   8  10  11  16  21  22  25  34  40  42  49  53  56  62  63  66  78
  91  97  98 101 104 117 119 121 123 128 130 132 140 141 143 145 161 168
 172 188 190 191 200 201 203 216 226 227]
[  1  14  18  20  29  31  50  55  57  59  67  72  74  83  84  86  92  93
  96 100 103 109 116 120 122 131 144 146 150 151 163 164 173 179 189 195
 196 198 199 202 206 210 217 221 224 225]


In [15]:
weight = dfsMLP.selected_ws[0]
index = np.argsort(abs(weight))[::-1]

In [32]:
for i in xrange(6):
    print(len(inputY[inputY==i]))

129
101
129
101
129
101


In [16]:
def topDFSfeatures(a,b):
    return X_train[:, index.tolist()[a:b]], X_test[:, index.tolist()[a:b]]

In [26]:
from sklearn.cross_validation import cross_val_score
from sklearn.svm import LinearSVC

svm = LinearSVC()

keke, keke_test = topDFSfeatures(0,150)
scores = cross_val_score(svm, keke, y_train, cv=5)
print(np.mean(scores))

0.939955085306


In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb

top_train, top_test = topDFSfeatures(0,500)

rf = RandomForestClassifier(criterion="entropy", n_estimators = 300, max_depth = 5)
rf.fit(top_train, y_train)

y_pred = rf.predict(top_test)

print(accuracy_score(y_test, y_pred))

gbm = xgb.XGBClassifier(max_depth=5, n_estimators=400, learning_rate=0.05).fit(top_train, y_train)
y_pred = gbm.predict(top_test)

featurescores = gbm.feature_importances_

print(accuracy_score(y_test, y_pred))

# gbm10 = xgb.XGBClassifier(max_depth=3, n_estimators=400, learning_rate=0.05).fit(X_train10, y_train)
# y_pred10 = gbm.predict(X_test10)

# print(accuracy_score(y_test, y_pred))

0.905797101449
0.898550724638


In [13]:
# X_train, X_test, y_train, y_test = train_test_split(inputX, inputY, test_size=0.2, random_state=1)

top_train, top_test = topDFSfeatures(20000,30000)

dfsMLP = DeepFeatureSelectionNew(top_train, top_test, y_train, y_test, n_input=1, hidden_dims=[100], dropout=[False], \
                                 learning_rate=0.01, lambda1=0.001, lambda2=1, alpha1=0.01, alpha2=0, activation='tanh', weight_init='uniform', \
                                 epochs=100, optimizer='Adam', print_step=5)
dfsMLP.train(batch_size=200)

In [1]:
import numpy as np
from scipy.sparse import issparse
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import check_array
from sklearn.utils.extmath import safe_sparse_dot


from math import log

def I(p,n):
	pos = p / (p + n)
	neg = n / (p + n)
	
	if pos == 0 and neg == 0:
		return 0
	elif pos == 0:
		return -neg * log(neg, 2)
	elif neg == 0:
		return -pos * log(pos, 2)
	else:
		return (-pos * log(pos, 2)) + (-neg * log(neg, 2))